In [19]:
from translator import translate
from transformers import pipeline
import pandas as pd
import numpy as np
from src.functions import check,tresh,license_plate
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc


In [23]:

df=pd.read_excel('./data_test/personal_data_db.xlsx')
df=df.dropna()
df['label']=df['label'].astype(int)
df

,sentence,label
0,"Je m’appelle Laval Jacquin, j’ai 38 ans, mon n...",1
1,Nour Kired et Damien Sonneville sont de bons a...,1
2,Arnaud Mimart est le chef de la chocoPolice,1
3,yeah that's what i i just did today i got uh D...,0
4,"Fondé en 1861, le Massachusetts Institute of T...",0
...,...,...
409,"In 1972, Phillip Morris, Inc.'s Miller Brewing...",0
410,"Ici, vous pourrez trouver en abondance des exe...",0
411,Bien que les membres du comité CVR ont considé...,0
412,Le DSI de l'entreprise travaille avec les resp...,0


In [2]:
# candidate_labels = ['First and last name', 'Mail','Internet protocol (IP) addresses', 'location', 'Date of birth', 'Date of death', 'Place of birth', 'Place of death', 'IP', 'a cookie', 'phone', 'health data','private data','personal data','not personal data','not private data']
name=["person's name","person"]
birth=["birthday","private"]
# ip=["Internet protocol (IP) addresses","IP adress"]
cookie=["cookie identifiers","a cookie"]
phone=["phone numbers","phone number","telphone number"]
mail=["mail addresses","mail address","mail"]
location=["location adress","private postal address"]
health=["health data","medical records","health information","medical information","Medical Procedure names","Medical Procedure identification codes"]
personal=["personal data","personal","personal information","private data","private information"]
not_personal=["not personal data","not personal information","not personal","not private data","not private","not private information"]
passport=['passport','code']
Driving_license=['driving license','code']
social_security_number=['social security number']
Tax_file_number=['tax file number']
credit_card=['credit card number']
candidate_labels=name+cookie+phone+mail+location+health+personal+not_personal+birth+passport+Driving_license+social_security_number+Tax_file_number+credit_card


In [3]:
pii_=[(name,'person'),(birth,'birth'),(cookie,'cookie'),(phone,'phone'),(mail,'mail'),(location,'location'),(health,'health'),(personal,'personal'),
        (not_personal,'not_personal'),(passport,"passport"),(Driving_license,"Driving_license"),(social_security_number,"social_security_number"),(Tax_file_number,"Tax_file_number"),(credit_card,"credit_card")]

In [4]:
pipeline = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

def predict(sentence : str,treshold : float):
    text_translated=translate(sentence)
    result_details=pipeline(text_translated, candidate_labels, multi_label=True)
    result_details=dict(zip(result_details['labels'],result_details['scores']))
    result = {name : np.mean(list(map(result_details.get, lst))) for lst,name in pii_}
    pii_detected={name : result[name] for name in tresh(result,treshold)}
    pii_detected=check(pii_detected,sentence)
    if license_plate(sentence):
            pii_detected["license_plate"]=0.9
    if pii_detected=={}:
            mean=0
    else:
            mean=np.mean(list(pii_detected.values()))
    del text_translated,result_details,result
    return int(pii_detected!={}),mean



In [6]:
y_pred=[]
predicted_proba=[]
for sent in tqdm(df['sentence']):
    result=predict(sent)
    y_pred.append(result[0])
    predicted_proba.append(result[1])

 30%|███       | 126/414 [12:16<28:24,  5.92s/it]

birth checked


 62%|██████▏   | 255/414 [24:53<15:39,  5.91s/it]

birth checked


100%|██████████| 414/414 [40:46<00:00,  5.91s/it]


In [7]:
conf = confusion_matrix(df['label'], y_pred)
cf = pd.DataFrame(conf, columns=['prédit ' + _ for _ in ['0','1']])
cf.index = ['vrai ' + _ for _ in ['0','1']]
cf

,prédit 0,prédit 1
vrai 0,289,103
vrai 1,8,14


In [11]:
np.mean([df.label==y_pred])

0.7318840579710145

In [9]:
result=df.copy()
result['predicted']=y_pred
result_no_match=result[result.label!=y_pred]
result_no_match.to_csv('./data_test/result_no_match.csv')
result_no_match

,sentence,label,predicted
8,"Je suis euh, grand sergent chef, à la retraite...",0,1
9,The FBI legal attach�'s office in Paris first ...,0,1
15,"In Prinsengracht, Otto Frank and his family hi...",0,1
22,He joined the rest of his team at their hotel.,0,1
24,"Je mange réellement vite, aussi vite que je pe...",0,1
...,...,...,...
392,Davidson should not adopt the pronunciation of...,0,1
394,"Et je pensais que c'était un privilège, et ça ...",0,1
403,You may have heard of me. Captain Calverley ...,0,1
404,"Pour plus de détails, voir //www.healtheffects...",0,1


In [5]:
result_no_match=pd.read_csv('./data_test/result_no_match.csv')
y_pred_06=[]
predicted_proba=[]
for sent in tqdm(result_no_match['sentence']):
    result=predict(sent,treshold=0.6)
    y_pred_06.append(result[0])
    predicted_proba.append(result[1])

100%|██████████| 111/111 [16:14<00:00,  8.78s/it]


0.44144144144144143

In [8]:
conf = confusion_matrix(result_no_match['label'], y_pred_06)
cf = pd.DataFrame(conf, columns=['prédit ' + _ for _ in ['0','1']])
cf.index = ['vrai ' + _ for _ in ['0','1']]
cf

,prédit 0,prédit 1
vrai 0,49,54
vrai 1,8,0


In [11]:
result_no_match=pd.read_csv('./data_test/result_no_match.csv')
y_pred_06=[]
predicted_proba=[]
for sent in tqdm(result_no_match['sentence']):
    result=predict(sent,treshold=0.7)
    y_pred_06.append(result[0])
    predicted_proba.append(result[1])

np.mean([result_no_match.label==y_pred_06])

100%|██████████| 111/111 [15:27<00:00,  8.36s/it]


0.6216216216216216

In [12]:
conf = confusion_matrix(result_no_match['label'], y_pred_06)
cf = pd.DataFrame(conf, columns=['prédit ' + _ for _ in ['0','1']])
cf.index = ['vrai ' + _ for _ in ['0','1']]
cf

,prédit 0,prédit 1
vrai 0,69,34
vrai 1,8,0


In [13]:
result_no_match=pd.read_csv('./data_test/result_no_match.csv')
y_pred_08=[]
predicted_proba=[]
for sent in tqdm(result_no_match['sentence']):
    result=predict(sent,treshold=0.8)
    y_pred_08.append(result[0])
    predicted_proba.append(result[1])

np.mean([result_no_match.label==y_pred_08])


100%|██████████| 111/111 [17:10<00:00,  9.28s/it]


0.6666666666666666

In [14]:
conf = confusion_matrix(result_no_match['label'], y_pred_08)
cf = pd.DataFrame(conf, columns=['prédit ' + _ for _ in ['0','1']])
cf.index = ['vrai ' + _ for _ in ['0','1']]
cf

,prédit 0,prédit 1
vrai 0,74,29
vrai 1,8,0


In [16]:
result_no_match=pd.read_csv('./data_test/result_no_match.csv')
y_pred_75=[]
predicted_proba=[]
for sent in tqdm(result_no_match['sentence']):
    result=predict(sent,treshold=0.9)
    y_pred_75.append(result[0])
    predicted_proba.append(result[1])

np.mean([result_no_match.label==y_pred_75])


100%|██████████| 111/111 [11:43<00:00,  6.34s/it]


0.7927927927927928

In [17]:
conf = confusion_matrix(result_no_match['label'], y_pred_75)
cf = pd.DataFrame(conf, columns=['prédit ' + _ for _ in ['0','1']])
cf.index = ['vrai ' + _ for _ in ['0','1']]
cf

,prédit 0,prédit 1
vrai 0,88,15
vrai 1,8,0
